In [ ]:
import json
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

seq_length = 20  # Number of time steps to look back
epochs = 100
batch_size = 40
data_size = 400

jsonData = json.load(open('dailyBTC1.json', encoding="utf-8"))
    
data = pd.DataFrame(jsonData['prices'][:data_size])
data.rename(columns={0: 'DateTime', 1: 'Price'}, inplace=True)

data.index = pd.to_datetime(data['DateTime'])
data = data.drop(columns=['DateTime'])


: 

In [ ]:
seq_length = 20  # Number of time steps to look back
epochs = 100
batch_size = 40
data_size = 400

# Normalizacja danych
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

In [ ]:
# Podział danych na zbiory treningowe i testowe
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

In [ ]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=(seq_length, 1)),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_absolute_error')
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)

In [ ]:
# Make predictions
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

# Inverse transform the predictions
train_predictions = scaler.inverse_transform(train_predictions)
test_predictions = scaler.inverse_transform(test_predictions)


In [ ]:
# Plot predictions
plt.figure(figsize=(10, 6))

# Plot actual data
plt.plot(data.index[seq_length:len(data)-seq_length], data['Price'][seq_length:len(data)-seq_length], label='Actual', color='blue')

# Plot training predictions
plt.plot(data.index[seq_length:seq_length+len(train_predictions)], train_predictions, label='Train Predictions', color='green')

# Plot testing predictions
test_pred_index = range(seq_length+len(train_predictions), seq_length+len(train_predictions)+len(test_predictions))
plt.plot(data.index[test_pred_index], test_predictions, label='Test Predictions', color='orange')

plt.xlabel('DateTime')
plt.ylabel('Price')
plt.legend()
plt.show()

Poniżej znajduje się szczegółowy opis kodu, który normalizuje dane, dzieli je na zbiory treningowe i testowe, tworzy sekwencje, oraz trenuje model LSTM przy użyciu biblioteki Keras:

### Normalizacja danych

Normalizacja danych jest ważnym krokiem w przetwarzaniu danych przed treningiem modelu, ponieważ przyspiesza i stabilizuje proces uczenia.

```python
from sklearn.preprocessing import MinMaxScaler

# Normalizacja danych
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)
```

#### Wyjaśnienie:
- **MinMaxScaler()**: Skaluje dane do zakresu [0, 1] (domyślnie).
- **fit_transform(data)**: Dopasowuje skalera do danych i jednocześnie przekształca dane.

#### Wzór:
Jeśli \( x \) to oryginalna wartość, \( x' \) to znormalizowana wartość, \( \text{min} \) i \( \text{max} \) to minimalne i maksymalne wartości w danych, to normalizacja odbywa się według wzoru:
\[ x' = \frac{x - \text{min}}{\text{max} - \text{min}} \]

### Podział danych na zbiory treningowe i testowe

Dzielimy dane na zbiory treningowe i testowe, przy czym 80% danych jest używane do trenowania, a 20% do testowania.

```python
# Podział danych na zbiory treningowe i testowe
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]
```

#### Wyjaśnienie:
- **train_size**: Liczba próbek w zbiorze treningowym (80% całego zestawu danych).
- **train_data**: Zbiór treningowy zawierający pierwsze 80% danych.
- **test_data**: Zbiór testowy zawierający pozostałe 20% danych.

### Tworzenie sekwencji

Sekwencje danych są tworzone do trenowania modelu LSTM.

```python
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 10  # przykładowa długość sekwencji
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)
```

#### Wyjaśnienie:
- **create_sequences(data, seq_length)**: Funkcja do tworzenia sekwencji danych i etykiet.
  - **X**: Lista sekwencji wejściowych.
  - **y**: Lista wartości docelowych (etykiet).
- **seq_length**: Długość każdej sekwencji.
- **np.array()**: Konwersja list na tablice numpy.

### Model LSTM

Tworzenie i kompilacja modelu LSTM.

```python
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=(seq_length, 1)),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_absolute_error')
```

#### Wyjaśnienie:
- **tf.keras.Sequential**: Inicjalizuje sekwencyjny model Keras.
- **tf.keras.layers.LSTM(50, activation='relu', input_shape=(seq_length, 1))**: Dodaje warstwę LSTM z 50 jednostkami i funkcją aktywacji ReLU. **input_shape** określa kształt wejściowy (długość sekwencji, liczba cech).
- **tf.keras.layers.Dense(1)**: Dodaje w pełni połączoną warstwę wyjściową z jedną jednostką.
- **compile(optimizer='adam', loss='mean_absolute_error')**: Kompiluje model z optymalizatorem Adam i funkcją straty MAE.

### Trening modelu

Trening modelu na zbiorze treningowym.

```python
epochs = 20  # przykładowa liczba epok
batch_size = 32  # przykładowa wielkość partii

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)
```

#### Wyjaśnienie:
- **fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)**: Trenuje model na danych treningowych.
  - **epochs**: Liczba epok treningowych.
  - **batch_size**: Wielkość partii danych.
  - **verbose**: Poziom szczegółowości wypisywanych informacji podczas treningu (1 = szczegółowe informacje).

Poniżej jest pełny kod:

```python
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Generowanie danych
data = np.random.random((1000, 1))  # przykładowe dane

# Normalizacja danych
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

# Podział danych na zbiory treningowe i testowe
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Funkcja do tworzenia sekwencji
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 10  # przykładowa długość sekwencji
X_train, y_train = create_sequences(train_data, seq_length)
X_test, y_test = create_sequences(test_data, seq_length)

# Tworzenie modelu LSTM
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, activation='relu', input_shape=(seq_length, 1)),
    tf.keras.layers.Dense(1)
])

# Kompilacja modelu
model.compile(optimizer='adam', loss='mean_absolute_error')

# Trening modelu
epochs = 20  # przykładowa liczba epok
batch_size = 32  # przykładowa wielkość partii

model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1)
```